In [1]:
#coding=utf-8
import sys
import numpy as np
from common import *
from scipy.io import loadmat
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from joint_bayesian import *
import tensorflow as tf
import numpy as np
import argparse
import facenet
import detect_face
import os
import sys
import math
import pickle
from sklearn.svm import SVC

In [2]:
with tf.Graph().as_default():

    with tf.Session() as sess:

        datadir = 'actor_136/'
        dataset = facenet.get_dataset(datadir)
        paths, labels = facenet.get_image_paths_and_labels(dataset)
        print('Number of classes: %d' % len(dataset))
        print('Number of images: %d' % len(paths))

        print('Loading feature extraction model')
        modeldir = 'parameter/20180402-114759/20180402-114759.pb'
        facenet.load_model(modeldir)

        images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]

        # Run forward pass to calculate embeddings
        print('Calculating features for images')
        batch_size = 10
        image_size = 160
        nrof_images = len(paths)
        nrof_batches_per_epoch = int(math.ceil(1.0 * nrof_images / batch_size))
        emb_array = np.zeros((nrof_images, embedding_size))
        epoch = 1

        for k in range(epoch):
            print(k)
            for i in range(nrof_batches_per_epoch):
                start_index = i * batch_size
                end_index = min((i + 1) * batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                images = facenet.load_data(paths_batch, False, False, image_size)
                feed_dict = {images_placeholder: images, phase_train_placeholder: False}
                emb_array[start_index:end_index, :] = sess.run(embeddings, feed_dict=feed_dict)
        print(emb_array)

Number of classes: 136
Number of images: 10824
Loading feature extraction model
Model filename: parameter/20180402-114759/20180402-114759.pb
Calculating features for images
0
[[ 0.00182328 -0.02977761 -0.04932759 ... -0.05696321 -0.01685259
   0.01652232]
 [ 0.07165083 -0.00872429 -0.00922326 ... -0.01227255 -0.03999925
  -0.01334668]
 [ 0.1034187  -0.00802998  0.0019207  ... -0.00513172 -0.04029639
  -0.01521532]
 ...
 [ 0.03613141 -0.0399299  -0.04435496 ...  0.02301129 -0.0073959
   0.01649485]
 [ 0.02712402 -0.10455494 -0.02683702 ... -0.06066393 -0.09218856
  -0.02170003]
 [ 0.01641258 -0.00865528 -0.07319351 ...  0.00871961 -0.02030928
   0.03157901]]


In [3]:
import pandas as pd

In [4]:
datas = pd.DataFrame(emb_array)
label = pd.DataFrame(labels, columns=['label'])
results = pd.concat([datas, label], axis=1)

In [5]:
res = results.groupby(['label']).mean()

In [6]:
with open('./JB_result/mean_emb_result.pkl', "wb") as f:
    pickle.dump(res.values, f)